In [1]:
!pip install aicrowd-cli==0.1
API_KEY = "4c49d255257272f8caf90b1e74b8cccd"
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [2]:
!ls

data	     sample_submission.csv  train.csv  val.csv
sample_data  test.zip		    train.zip  val.zip


In [3]:
!aicrowd dataset download --challenge rover-classification -j 3

sample_submission.csv:   0% 0.00/164k [00:00<?, ?B/s]
train.csv:   0% 0.00/689k [00:00<?, ?B/s]

sample_submission.csv: 100% 164k/164k [00:00<00:00, 385kB/s]


sample_submission.csv: 100% 164k/164k [00:00<00:00, 384kB/s]
train.csv: 100% 689k/689k [00:00<00:00, 937kB/s]
train.csv: 100% 689k/689k [00:00<00:00, 936kB/s]

train.zip:   0% 0.00/266M [00:00<?, ?B/s]

val.csv:   0% 0.00/64.8k [00:00<?, ?B/s]

val.csv: 100% 64.8k/64.8k [00:00<00:00, 264kB/s]


test.zip:  51% 33.6M/66.4M [00:02<00:02, 13.0MB/s]
test.zip: 100% 66.4M/66.4M [00:04<00:00, 14.6MB/s]


test.zip: 100% 66.4M/66.4M [00:04<00:00, 15.8MB/s]

val.zip: 100% 26.6M/26.6M [00:02<00:00, 11.9MB/s]
train.zip: 100% 266M/266M [00:13<00:00, 19.2MB/s]


In [4]:
!rm -rf data
!mkdir data


!unzip train.zip -d data/train >/dev/null
!unzip val.zip -d data/val >/dev/null
!unzip test.zip -d data/test >/dev/null

In [5]:
import pandas as pd
import os
import tensorflow as tf

In [6]:
df_train = pd.read_csv("train.csv")

In [7]:
df_val = pd.read_csv("val.csv")

In [8]:
df_train['ImageID'] = df_train['ImageID'].astype(str)+".jpg"
df_val['ImageID'] = df_val['ImageID'].astype(str)+".jpg"

In [9]:
INPUT_SIZE = 256

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
len(df_train), len(df_val)

(40000, 4000)

In [12]:
datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
dataframe=df_train,
directory="data/train/",
x_col="ImageID",
y_col="label",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 40000 validated image filenames belonging to 2 classes.


In [13]:
val_generator=datagen.flow_from_dataframe(
dataframe=df_val,
directory="data/val/",
x_col="ImageID",
y_col="label",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 4000 validated image filenames belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers, optimizers
import os
import numpy as np
import pandas as pd

In [15]:
class CustomAugment(object):
    def __call__(self, image):        
        # Random flips and grayscale with some stochasticity
        img = self._random_apply(tf.image.flip_left_right, image, p=0.5)
        img = self._random_apply(self._color_drop, img, p=0.8)
        return img

    def _color_drop(self, x):
        image = tf.image.rgb_to_grayscale(x)
        image = tf.tile(x, [1, 1, 1, 3])
        return x
    
    def _random_apply(self, func, x, p):
        return tf.cond(
          tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                  tf.cast(p, tf.float32)),
          lambda: func(x),
          lambda: x)

In [16]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.Lambda(CustomAugment()),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(INPUT_SIZE, 
                                                              INPUT_SIZE,
                                                              3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [17]:
model = Sequential()
model.add(data_augmentation)
model.add(tf.keras.applications.MobileNetV2(input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizers.RMSprop(lr=0.0001/10), loss="categorical_crossentropy", metrics=["Recall", "Precision"])

In [18]:
model.layers[1].trainable = False

In [19]:
next(train_generator)[0].shape

(64, 256, 256, 3)

In [20]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VAL=val_generator.n//train_generator.batch_size

In [21]:
model.fit(train_generator, validation_data=val_generator, epochs=7)

Epoch 1/7
625/625 [==============================] - 210s 322ms/step - loss: 0.5998 - recall: 0.7725 - precision: 0.7725 - val_loss: 1.6361 - val_recall: 0.5740 - val_precision: 0.5740
Epoch 2/7
625/625 [==============================] - 200s 320ms/step - loss: 0.1219 - recall: 0.9563 - precision: 0.9563 - val_loss: 1.4531 - val_recall: 0.6720 - val_precision: 0.6720
Epoch 3/7
625/625 [==============================] - 200s 319ms/step - loss: 0.0667 - recall: 0.9786 - precision: 0.9786 - val_loss: 0.5831 - val_recall: 0.8590 - val_precision: 0.8590
Epoch 4/7
625/625 [==============================] - 199s 319ms/step - loss: 0.0497 - recall: 0.9854 - precision: 0.9854 - val_loss: 0.2457 - val_recall: 0.9503 - val_precision: 0.9503
Epoch 5/7
625/625 [==============================] - 200s 319ms/step - loss: 0.0317 - recall: 0.9907 - precision: 0.9907 - val_loss: 0.0578 - val_recall: 0.9860 - val_precision: 0.9860
Epoch 6/7
625/625 [==============================] - 197s 315ms/step - loss

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, None, None, None)  0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 8, 8, 1280)        2257984   
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 81920)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               41943552  
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)              

In [23]:
model.layers[1].trainable = True

In [24]:
tf.keras.backend.set_value(model.optimizer.learning_rate, 0.0001/100)

In [25]:
model.fit(train_generator, validation_data=val_generator, epochs=3)

Epoch 1/3
625/625 [==============================] - 200s 319ms/step - loss: 0.0160 - recall: 0.9956 - precision: 0.9956 - val_loss: 0.0371 - val_recall: 0.9910 - val_precision: 0.9910
Epoch 2/3
625/625 [==============================] - 199s 319ms/step - loss: 0.0153 - recall: 0.9960 - precision: 0.9960 - val_loss: 0.0404 - val_recall: 0.9902 - val_precision: 0.9902
Epoch 3/3
625/625 [==============================] - 200s 319ms/step - loss: 0.0115 - recall: 0.9969 - precision: 0.9969 - val_loss: 0.0524 - val_recall: 0.9890 - val_precision: 0.9890


In [26]:
df_test = pd.read_csv("sample_submission.csv",dtype=str)
df_test["ImageID"] = df_test["ImageID"].astype(str)+".jpg"

In [27]:
test_generator=datagen.flow_from_dataframe(
dataframe=df_test,
directory="data/test/",
x_col="ImageID",
y_col="label",
batch_size=1,
seed=42,
shuffle=False,
class_mode="categorical",
target_size=(INPUT_SIZE,INPUT_SIZE))

Found 10000 validated image filenames belonging to 2 classes.


In [28]:
model

In [29]:
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size

In [30]:
STEP_SIZE_TEST

10000

In [31]:
test_generator.reset()
pred = model.predict(test_generator,
steps=STEP_SIZE_TEST, verbose=1)

10000/10000 [==============================] - 67s 7ms/step


In [32]:
predicted_class_indices = np.argmax(pred,axis=1)

In [33]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [34]:
df_test["pred"] = predictions

In [35]:
df_test.head()

,ImageID,label,pred
0,0.jpg,curiosity,curiosity
1,1.jpg,perseverance,curiosity
2,2.jpg,curiosity,curiosity
3,3.jpg,perseverance,perseverance
4,4.jpg,perseverance,curiosity


In [36]:
df_test.drop("label", axis=1, inplace=True)
df_test.rename(columns={"pred": "label"}, inplace=True)

In [37]:
df_test.head()

,ImageID,label
0,0.jpg,curiosity
1,1.jpg,curiosity
2,2.jpg,curiosity
3,3.jpg,perseverance
4,4.jpg,curiosity


In [38]:
import re

In [39]:
df_test["ImageID"] = df_test["ImageID"].map(lambda x: re.sub(r"\D", "", str(x)))

In [40]:
df_test.head()

,ImageID,label
0,0,curiosity
1,1,curiosity
2,2,curiosity
3,3,perseverance
4,4,curiosity


In [41]:
df_test.to_csv("data/02.2_sub.csv", index=False)

In [42]:
!aicrowd submission create -c rover-classification -f data/02.2_sub.csv

02.2_sub.csv ━━━━━━━━━━━━━━━━━━━━ 100.0% • 165.3/163.7 KB • 289.1 kB/s • 0:00:00
                                    ╭─────────────────────────╮                                     
                                    │ Successfully submitted! │                                     
                                    ╰─────────────────────────╯                                     
                                          Important links                                           
┌──────────────────┬───────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-7/submissions/126625              │
│                  │                                                                               │
│  All submissions │ https://www.aicrowd.com/challenges/ai-blitz-7/submissions?my_submissions=true │
│                  │                                                                               │
│      Lea